### This scripts takes about 1 1/2 minutes seconds to execute

This idea of this notebook is to create a first (simple) DL model using all the same features used in the multinomial logit model.

Much inspiration was derived from https://towardsdatascience.com/use-machine-learning-to-predict-horse-racing-4f1111fb6ced.

In [18]:
import math
from importlib import reload
import deeplearninglib
reload(deeplearninglib)
from deeplearninglib import *

import wandb

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [19]:
# select model to train

model_inventory = {'mktprob': {'XZ_columns': ["mkt_prob"],
                               'continuous_features': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': LinSig},
                   'mktprob_soft': {'XZ_columns': ["mkt_prob"],
                               'continuous_features': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': LinSoft},
                    'mktprob_MLR': {'XZ_columns': ["mkt_prob"],
                               'continuous_features': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': MLR},
                   'AlunOwen_v0': {'XZ_columns': ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v1': {'XZ_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v2': {'XZ_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinDropReluLinSoft},
                   'AlunOwen_v3': {'XZ_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': ["age", "trainer_sr", "daysLTO"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': False,
                                   'model_architecture': MLR},
                   'AlunOwen_v4': {'XZ_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie"],
                                   'continuous_features': [],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': False,
                                   'model_architecture': MLR},
                   'test_v1': {'XZ_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "cheekpieces", "tonguetie", "course_Kempton", "course_Lingfield", "course_Southwell", "course_Wolverhampton", "going_Firm", "going_Good", "going_Good to Firm", "going_Good to Soft", "going_Slow", "going_Soft", "going_Standard", "going_Standard to Slow", "direction_Left Handed", "direction_Right Handed"],
                                   'continuous_features': [],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': False,
                                   'model_architecture': LinDropReluLinSoft}
                               }

model = 'AlunOwen_v3'
XZ_columns = model_inventory[model]['XZ_columns']
continuous_features = model_inventory[model]['continuous_features']
learning_rate = model_inventory[model]['learning_rate']
epochs = model_inventory[model]['epochs']
vacant_stall_indicator = model_inventory[model]['vacant_stall_indicator']
bias = model_inventory[model]['bias']
model_architecture = model_inventory[model]['model_architecture']

In [20]:
# read in data

y_columns = ["win"] # assumed to be contained in runners files

runners_train_data_fn = "data\\runners_train.csv"
races_train_data_fn = "data\\races_train.csv"
runners_validate_data_fn = "data\\runners_validate.csv"
races_validate_data_fn = "data\\races_validate.csv"

train_data = RacesDataset(runners_train_data_fn, races_train_data_fn, XZ_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator, continuous_features=continuous_features)
validate_data = RacesDataset(runners_validate_data_fn, races_validate_data_fn, XZ_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator, scalar=train_data.scalar, continuous_features=continuous_features)

train_dataloader = DataLoader(train_data, batch_size=64)
validate_dataloader = DataLoader(validate_data, batch_size=64)

In [21]:
# prepare and save test data for use by Dividend Project Model; then delete them to ensure they're not used during training or validation

# note, test data is model-specific

runners_test_data_fn = "data\\runners_test.csv"
races_test_data_fn = "data\\races_test.csv"

test_data = RacesDataset(runners_test_data_fn, races_test_data_fn, XZ_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator, scalar=train_data.scalar, continuous_features=continuous_features)

torch.save(test_data, "data\\" + model + "_test_data.pt")

del test_data

In [22]:
pd.options.display.max_columns = 1000 # was 20
train_data.runners_wide.loc[:, train_data.X_columns].head()

age                                                    \
stall_number        1         2         3         4         5         6    
race_id                                                                    
11504        -0.671572 -0.671572 -0.671572 -0.671572 -0.671572 -0.671572   
11505        -0.140729  0.920958  2.513488  1.451801  0.390115  1.982644   
11506        -0.671572 -0.671572 -0.671572 -0.671572 -0.671572 -0.671572   
11507         0.390115 -0.140729 -0.140729 -0.140729 -0.140729 -0.140729   
11508         1.451801 -0.140729  0.390115 -0.140729  0.390115 -0.140729   

                                                                               \
stall_number        7         8         9         10        11   12   13   14   
race_id                                                                         
11504        -0.671572 -0.671572  0.000000  0.000000  0.000000  0.0  0.0  0.0   
11505         0.390115  0.390115  0.390115 -0.140729  0.390115  0.0  0.0  0.0   
11506        -0.671572 -0.671572 -0.671572 -0.671572  0.000000  0.0  0.0  0.0   
11507         1.982644 -0.140729 -0.140729  0.000000  0.000000  0.0  0.0  0.0   
11508         0.390115  0.390115 -0.140729  0.000000  0.000000  0.0  0.0  0.0   

                       blinkers                                               \
stall_number   15   16       1    2    3    4    5    6    7    8    9    10   
race_id                                                                        
11504         0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0      1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0   
11507         0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                           cheekpieces                      \
stall_number   11   12   13   14   15   16          1    2    3    4    5    
race_id                                                                      
11504         0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  1.0  0.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0         0.0  1.0  1.0  0.0  0.0   

                                                                      daysLTO  \
stall_number   6    7    8    9    10   11   12   13   14   15   16        1    
race_id                                                                         
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.638658   
11505         0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  1.638658   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.638658   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.638658   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.638658   

                                                                          \
stall_number        2         3         4         5         6         7    
race_id                                                                    
11504         1.638658  1.638658  1.638658  1.638658  1.638658  1.638658   
11505         1.638658  1.638658  1.638658  1.638658  1.638658  1.638658   
11506         1.638658  1.638658  1.638658  1.638658  1.638658  1.638658   
11507         1.638658  1.638658  1.638658  1.638658  1.638658  1.638658   
11508         1.638658  1.638658  1.638658  1.638658  1.638658  1.638658   

                                                                               \
stall_number        8         9         10        11   12   13   14   15   16   
race_id                                                                    

In [23]:
assert model_architecture != "MLR" or train_data.races is None, "Use of MLR with race-specific variables isn't yet supported"
train_data.races.head()

""
race_id
11504
11505
11506
11507
11508


In [24]:
# build the neural network

output_layer_nodes = train_data.y.shape[1]
if train_data.Z is not None:
    input_layer_nodes = train_data.X.shape[1] + train_data.Z.shape[1]
else:
    input_layer_nodes = train_data.X.shape[1]

torch.manual_seed(0)
net = model_architecture(input_layer_nodes, output_layer_nodes, bias=bias).to(device) # linear-relu-linear-softwax nn (1 hidden layer)
print(f"Model structure: {model}")

for name, param in net.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param} \n")

Model structure: AlunOwen_v3
Layer: neural_network.0.weights | Size: torch.Size([20]) | Values : Parameter containing:
tensor([-0.0040,  0.2867, -0.4399, -0.3934, -0.2059,  0.1433, -0.0106,  0.4238,
        -0.0474,  0.1414, -0.1615, -0.1051, -0.5107, -0.3540, -0.2203,  0.0198,
         0.2113,  0.3207, -0.3624, -0.2328], requires_grad=True) 



In [25]:
# example to show how model is used from prediction

inputs = torch.rand(1, input_layer_nodes, device=device)
logits = net(inputs)
y_pred = logits.argmax(1)
print(f"Predicted class: {y_pred}")
print(inputs.shape)

Predicted class: tensor([14])
torch.Size([1, 320])


In [26]:
%env WANDB_NOTEBOOK_NAME 'C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="horse-racing-project",
    
    # track hyperparameters and run metadata
    config={
    "device": device,
    "model": model,
    "XZ_columns": XZ_columns,
    "continuous_featurs": continuous_features,
    "learning_rate": learning_rate,
    "epochs": epochs,
    "vacant_stall_indicator": vacant_stall_indicator,
    "bias": bias,
    "model_architecture": list(net.modules())
    }
)

env: WANDB_NOTEBOOK_NAME='C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'


In [27]:
# optimizing model parameters

# initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net, loss_fn, optimizer, device)
    (acc, loss) = validate_loop(validate_dataloader, net, loss_fn, device)
    wandb.log({"acc": acc, "loss": loss})
print("Done!")

Epoch 1
-------------------------------
loss: 2.776936  [   64/15874]
loss: 2.763041  [ 6464/15874]
loss: 2.803309  [12864/15874]
Validation Error:
 Accuracy: 10.4%, Avg loss: 2.778028 

Epoch 2
-------------------------------
loss: 2.776970  [   64/15874]
loss: 2.760748  [ 6464/15874]
loss: 2.800752  [12864/15874]
Validation Error:
 Accuracy: 11.0%, Avg loss: 2.776055 

Epoch 3
-------------------------------
loss: 2.776941  [   64/15874]
loss: 2.758337  [ 6464/15874]
loss: 2.798066  [12864/15874]
Validation Error:
 Accuracy: 11.9%, Avg loss: 2.773952 

Epoch 4
-------------------------------
loss: 2.776816  [   64/15874]
loss: 2.755800  [ 6464/15874]
loss: 2.795234  [12864/15874]
Validation Error:
 Accuracy: 12.6%, Avg loss: 2.771704 

Epoch 5
-------------------------------
loss: 2.776550  [   64/15874]
loss: 2.753131  [ 6464/15874]
loss: 2.792241  [12864/15874]
Validation Error:
 Accuracy: 13.3%, Avg loss: 2.769292 

Epoch 6
-------------------------------
loss: 2.776078  [   64/15

In [28]:
# finish the wandb run, necessary in notebooks
wandb.finish()

acc,▁▂▃▄▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
loss,██▇▇▆▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.22132
loss,2.66492


In [29]:
for para_name, para_vals in net.named_parameters():
    np.savetxt("weights and biases\\" + para_name + ".csv", para_vals.data.numpy(), fmt='%6.3f', delimiter=",")

In [30]:
# save trained model for use by Dividend Project Model
torch.save(net, "models\\" + model + ".pt")